<a href="https://colab.research.google.com/github/Anshad-Aziz/Machine_Learning/blob/main/Next_Words_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
file=open('/content/Sherlock Holmes.txt','r',encoding="utf8")

In [3]:
lines=[]
for i in file:
  lines.append(i)

In [4]:
data=""
for i in lines:
  data=' '.join(lines)

In [5]:
data=data.replace('\n','').replace('\r','').replace('\ufeff','').replace('”','')

In [6]:
data=data.split()
data=' '.join(data)

In [7]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [8]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])

In [10]:
pickle.dump(tokenizer,open('token.pkl','wb'))

In [11]:
sequence_data=tokenizer.texts_to_sequences([data])[0]

In [12]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [13]:
len(sequence_data)

105879

In [14]:
vocab_size=len(tokenizer.word_index)+1

In [15]:
vocab_size

8200

In [16]:
sequence=[]
for i in range(3,len(sequence_data)):
  words=sequence_data[i-3:i+1]
  sequence.append(words)

In [17]:
len(sequence)

105876

In [18]:
sequence=np.array(sequence)

In [19]:




sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [20]:
x=[]
y=[]
for i in sequence:
  x.append(i[0:3])
  y.append(i[3])

In [21]:
x=np.array(x)
y=np.array(y)

In [22]:




y=to_categorical(y,num_classes=vocab_size)

In [23]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation='relu'))
model.add(Dense(vocab_size,activation="softmax"))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             82000     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8200)              8208200   
                                                                 
Total params: 21339200 (81.40 MB)
Trainable params: 21339200 (81.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint=ModelCheckpoint("next_words.h5",monitor="loss",verbose=1,save_best_only=True)
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001))
model.fit(x,y,epochs=2,batch_size=64,callbacks=[checkpoint])

Epoch 1/2
1655/1655 [==============================] - ETA: 0s - loss: 6.3441
Epoch 1: loss improved from inf to 6.34412, saving model to next_words.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1655/1655 [==============================] - 45s 22ms/step - loss: 6.3441
Epoch 2/2
1655/1655 [==============================] - ETA: 0s - loss: 5.7617
Epoch 2: loss improved from 6.34412 to 5.76174, saving model to next_words.h5
1655/1655 [==============================] - 30s 18ms/step - loss: 5.7617
